In [26]:
import sys
import itertools
from anytree import Node, RenderTree,Resolver,ChildResolverError,NodeMixin,PreOrderIter
from anytree.dotexport import RenderTreeGraph

class CustomNode(NodeMixin):  # Add state number feature
    def __init__(self, name, state_number,symbol,end_state, parent=None):
        super(CustomNode, self).__init__()
        self.name=name
        self.state_number=state_number
        self.parent=parent
        self.end_state=end_state
        self.symbol=symbol

state_number=0
top=CustomNode("root",state_number,'',False)
r = Resolver('name')




In [27]:
def add_inflectional_suffixes(suffix,state_number,root_node):
    suffixes=suffix.split(":")
    state_number+=1
    node=CustomNode('<eps>',state_number,'-',False,root_node)
    state_number+=1
    node=CustomNode('^',state_number,'<eps>',False,node)
    for ch in suffixes[0][1:]:
        state_number+=1
        node=CustomNode(ch,state_number,ch,False,node)
    state_number+=1
    CustomNode('<eps>',state_number,suffixes[1],True,node)

    if suffixes[1]=='+N':
        state_number+=1
        plural_node=CustomNode("^s",state_number,'+PL',True,node)
        state_number+=1
        CustomNode("'s",state_number,'+SGPS',True,node)
        state_number+=1
        CustomNode("'",state_number,'+PLPS',True,plural_node)
    elif suffixes[1]=='+V':
        state_number+=1
        CustomNode("^s",state_number,'+3D',True,node)
        state_number+=1
        CustomNode("^ing",state_number,'+ING',True,node)
    return (node,state_number)
def append_suffixes(root_node,morphs,index,state_number):
    suffix=''
    while len(morphs)>index:
        index+=1
        if morphs[index]=='|':
            if suffix!='': #check if suffix to be handle before pipe
                
                node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
                suffix=''
        elif morphs[index]=='(':
            node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
            index,state_number=append_suffixes(node,morphs,index,state_number)
            suffix=''
        elif morphs[index]==')':
            if suffix!='': #check if suffix to be handle before paranthesis
                val=add_inflectional_suffixes(suffix,state_number,root_node)
                state_number=val[1]
            return (index,state_number)
        else:
            suffix+=morphs[index]
    return (index,state_number)

def insert_irregular_noun(stem,state_number,root):
    top=root
    verbs=stems=stem.split('/')
    for i in range(len(verbs[0])):
        state_number+=1
        root=CustomNode(verbs[0][i],state_number,verbs[0][i],False,root)
    end_ptr=root
   
    
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+N',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+SG',True,end_ptr)
   
    end_ptr=top
    for i in range(len(verbs[0])):
        state_number+=1
        end_ptr=CustomNode('<eps>',state_number,verbs[0][i],False,end_ptr)
   
    
    for i in range(len(verbs[1])):
        state_number+=1
        end_ptr=CustomNode(verbs[1][i],state_number,'<eps>',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+N',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+PL',True,end_ptr)
            
    return (root,state_number)

In [28]:
#words = ['compute (er:+N(ize:+V(ation:+N|able:+ADJ)|less:+ADJ(ness:+N))|ation:+N(able:+ADJ))','work (er:+N(ism:+N))']
#words = ["be/were&was/been (ing:+N)","begin/began/begun (ing:+N|er:+N)"]
with open('Lexicon/Nouns.txt') as f:
    words = f.readlines()
    words.sort()
    for word in words:
        line=word.rstrip()
        print(line)
        arr=line.split(' ')
        stem=arr[0]
        if len(arr)==2:
            morphs=arr[1]
        root=top

        if '/' in stem: #means irregular verb
            root,state_number=insert_irregular_noun(stem,state_number,top)
        else:
            for i in range(len(stem)):
                try:
                    n=r.get(root,stem[i])
                except ChildResolverError:
                    state_number+=1
                    n=CustomNode(stem[i],state_number,stem[i],False,root)
                root=n
            
            state_number+=1
            end_point=CustomNode("<eps>",state_number,'+N',False,root)
            state_number+=1
            CustomNode("<eps>",state_number,'+SG',True,end_point)
            state_number+=1
            CustomNode("^s",state_number,'+PL',True,end_point)
        
        if len(arr)==2:
            return_val=append_suffixes(root,morphs,0,state_number)
            state_number=return_val[1]
RenderTreeGraph(top).to_picture("udo.png")

accident (-al:+ADJ(-ism:+N|-ist:+N|ness:+N|-ity:+N|-ly:+ADV))
act
aim
anger (-y:+ADJ(-ly:+ADV))
answer
anxiety (-ous:+ADJ(-ly:+ADV|-ness:+N))
art (-ist:+N(-ry:+N|-ic:+ADJ(-al:+ADJ))|-y:+ADJ)
attack
ban
cheer (-ful:+ADJ(-ly:+ADV|-ness:+N))
craze (-y:+ADJ(-ly:+ADV|-ness:+N))
data/data
doubt (-ful:+ADJ(-ly:+ADV|-ness:+N))
drama (-tic:+ADJ(-s:+N|-ally:+ADV))
duty (-ful:+ADJ(-ly:+ADV|-ness:+N))
enormity (-ous:+ADJ(-ly:+ADV|-ness:+N))
escape
event (-ful:+ADJ(-ness:+N)|-ual:+ADJ(ly:+ADV|-ity:+N)|-ly:+ADV)
faith (-ful:+ADJ(-ly:+ADV|-ness:+N))
final (-e:+N|-ly:+ADV)
fool (-ish:+ADJ(-ly:+ADV|-ness:+N))
glee (-ful:+ADJ(-ly:+ADV|-ness:+N))
grace (-ful:+ADJ(-ly:+ADV|-ness:+N))
haste (-n:+N(-ly:+ADV|-ness:+N))
hate
history (-ial:+ADJ|-ial:+N|-ian:+N|-iate:+V(-ed:+ADJ)|-ic:+ADJ(-al:+N|-ism:+N|-ist:+N|-ity:+N|-ize:+V(-ation:+N|-er:+N))|-ian:+N|-less:+ADJ)
hope (-less:+ADJ(-ly:+ADV|-ness:+N))
hour (-ly:+ADV)
innocent (-ce:+N|-ly:+ADV)
insult
kind (-ly:+ADJ)
meat (-al:+ADV|-us:+N|-less:+ADJ|-ed:+ADJ|-y:

In [29]:
p=PreOrderIter(top)

p=itertools.islice(p,1,None) #pass root


for n in p:
    print(n.name + " " +str(n.state_number) + "parent: " + n.parent.name)

a 1parent: root
c 2parent: a
c 3parent: c
i 4parent: c
d 5parent: i
e 6parent: d
n 7parent: e
t 8parent: n
<eps> 9parent: t
<eps> 10parent: <eps>
^s 11parent: <eps>
<eps> 12parent: t
^ 13parent: <eps>
a 14parent: ^
l 15parent: a
<eps> 16parent: l
<eps> 17parent: l
^ 18parent: <eps>
i 19parent: ^
s 20parent: i
m 21parent: s
<eps> 22parent: m
^s 23parent: m
' 25parent: ^s
's 24parent: m
<eps> 26parent: l
^ 27parent: <eps>
i 28parent: ^
s 29parent: i
t 30parent: s
<eps> 31parent: t
^s 32parent: t
' 34parent: ^s
's 33parent: t
<eps> 35parent: l
^ 36parent: <eps>
e 37parent: ^
s 38parent: e
s 39parent: s
<eps> 40parent: s
^s 41parent: s
' 43parent: ^s
's 42parent: s
<eps> 44parent: l
^ 45parent: <eps>
i 46parent: ^
t 47parent: i
y 48parent: t
<eps> 49parent: y
^s 50parent: y
' 52parent: ^s
's 51parent: y
<eps> 53parent: l
^ 54parent: <eps>
l 55parent: ^
y 56parent: l
<eps> 57parent: y
t 58parent: c
<eps> 59parent: t
<eps> 60parent: <eps>
^s 61parent: <eps>
i 62parent: a
m 63parent: i
<eps> 

In [30]:

#for pre, _, node in RenderTree(top):
#    treestr = u"%s%s" % (pre, node.name)
#    print(treestr.ljust(8), node.state_number)
def print_as_tabs(prev_state,next_state,input_label,output_label):
    print(str(prev_state) + "\t" + str(next_state) + "\t"+ str(input_label) + "\t" + str(output_label))


# In[222]:


p=PreOrderIter(top)
p=itertools.islice(p,1,None) #pass root

print_as_tabs(0,1,"<s>","<s>")
print_as_tabs(1,0,"<eps>","<eps>")
print_as_tabs(0,2,"</s>","</s>")
print_as_tabs(2,0,"<eps>","<eps>")
print_as_tabs(0,3,"<unk>","<unk>")
print_as_tabs(3,0,"<eps>","<eps>")
for n in p:
    parent_state_number=n.parent.state_number
    if n.parent.state_number!=0:
        parent_state_number+=3
    print_as_tabs(parent_state_number,n.state_number+3,n.symbol,n.name)
    if n.end_state:
        print_as_tabs(n.state_number+3,0,"<eps>","#")  
    #if n.is_leaf:
     #   print_as_tabs(n.state_number+3,0,"<eps>","#")
print("0")


0	1	<s>	<s>
1	0	<eps>	<eps>
0	2	</s>	</s>
2	0	<eps>	<eps>
0	3	<unk>	<unk>
3	0	<eps>	<eps>
0	4	a	a
4	5	c	c
5	6	c	c
6	7	i	i
7	8	d	d
8	9	e	e
9	10	n	n
10	11	t	t
11	12	+N	<eps>
12	13	+SG	<eps>
13	0	<eps>	#
12	14	+PL	^s
14	0	<eps>	#
11	15	-	<eps>
15	16	<eps>	^
16	17	a	a
17	18	l	l
18	19	+ADJ	<eps>
19	0	<eps>	#
18	20	-	<eps>
20	21	<eps>	^
21	22	i	i
22	23	s	s
23	24	m	m
24	25	+N	<eps>
25	0	<eps>	#
24	26	+PL	^s
26	0	<eps>	#
26	28	+PLPS	'
28	0	<eps>	#
24	27	+SGPS	's
27	0	<eps>	#
18	29	-	<eps>
29	30	<eps>	^
30	31	i	i
31	32	s	s
32	33	t	t
33	34	+N	<eps>
34	0	<eps>	#
33	35	+PL	^s
35	0	<eps>	#
35	37	+PLPS	'
37	0	<eps>	#
33	36	+SGPS	's
36	0	<eps>	#
18	38	-	<eps>
38	39	<eps>	^
39	40	e	e
40	41	s	s
41	42	s	s
42	43	+N	<eps>
43	0	<eps>	#
42	44	+PL	^s
44	0	<eps>	#
44	46	+PLPS	'
46	0	<eps>	#
42	45	+SGPS	's
45	0	<eps>	#
18	47	-	<eps>
47	48	<eps>	^
48	49	i	i
49	50	t	t
50	51	y	y
51	52	+N	<eps>
52	0	<eps>	#
51	53	+PL	^s
53	0	<eps>	#
53	55	+PLPS	'
55	0	<eps>	#
51	54	+SGPS	's
54	0	<eps>	#
18	56	-	<eps>
56	57	<eps>	^

In [ ]:
def append_suffixes(node_stack,morphs,index,paranthesis_stack,state_number):
    suffix=''
    while len(paranthesis_stack)!=0:
        index+=1
        
        if morphs[index]=='|':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
                suffix=''
        elif morphs[index]=='(':
            suffixes=suffix.split(':')
            state_number+=1
            
            node=CustomNode(suffixes[0],state_number,suffixes[1],node_stack[-1])
            node_stack.append(node)
            paranthesis_stack.append(index)
            param_node_stack=[node_stack[-1]]
            param_paranthesis_stack=[index]
            print(node_stack[-1].name)
            print(param_paranthesis_stack)
        
            var=append_suffixes(param_node_stack,morphs,index,param_paranthesis_stack,state_number)
            index=var[0]
            state_number=var[1]
            node_stack.pop()
            paranthesis_stack.pop()
            suffix=''
            print(node_stack[-1].name)
        elif morphs[index]==')':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
            paranthesis_stack.pop()
            node_stack.pop()
            return (index,state_number,node_stack,paranthesis_stack)
        else:
            suffix+=morphs[index]
    return (index,state_number,node_stack,paranthesis_stack)